# NYC fare prediction using Rapids and XG Boost running on GPU

[This notebook](TBD) shows my approach for predicting the fare amount for a taxi ride in NYC when given the pickup and dropoff locations of the passangers regarding the [New York City Taxi Fare Prediction Challange]( https://www.kaggle.com/c/new-york-city-taxi-fare-prediction).



---

This notebook is seperated into different sections, relating to the common data science workflow (except the hypothesis and data collection where already done). I tested different models to get used to Rapids and Cudf by starting with simple linear models and ended up with XG Boost and Light GBM. The best performant results are deffinitely the XG Boost ones. Different other Kernels state that they achieve similar results with LGBM hence I had problems with running the model on the GPU although I installed the right version (some bug reports also suggest there is a problem with the model).


0.   Previous Commits
1.   Setup and Check Infrastructure
2.   Having a first look at the Data (EDA)
3.   Data Cleaning (Feature Engineering)
4.   Linear Regression GPU
5.   Ridge Regression GPU
6.   K-Nearest Neighbor Regression GPU
7.   Random Forest GPU
8.   XG Boost on GPU
9.   Light GBM on GPU (not running on Colab)
10.   Stacked Ensemble XGB and LGBM
11.   Evaulation

---

## Approach:

I started with this project by getting used to the Rapids environment and the Cuda librarys and went on for more advanced models (XGB and LGBM) with testing them on common parameters mentioned in different notebooks on kaggle (see inspiration in first section) to get a feeling what influences what and how does it generelly work. After gaining some experience I combined several data cleaning approaches from differnt notebooks and general information from the internet. The most promising score yet was 3.19347 which would result in rank 432 of the public leaderboard (if the competition would still run). My kaggle notebook can be found [here, user handle: AlexS2020](https://)


## Evaluation:
As will be mentioned in the last section, I used to compare the different RMSE and R2 score values of different models as well as uploading the submission files to get a direct comperision via the score. Last step to evaluate are the confusion_matrices to see how many false positive/negative and true postivie/negative there are.

## Conclusion:
Overall it was an interesting project / competition which enabled me to learn new stuff and try out several tools. It is interesting to see, that the same code running on the GPU is so much faster compared to CPU implementations when looking at XGB for example. Interesting to me was also that relatively simple models like the KNN or Random Forest performed quite well in relation to the training runtime and ressource consumption. It would have been nice to directly compare the LBGM model against XGB since I read quite a bit about both. Unfortunately this was not possible within the desired scope due to runtime issues mostly. 

Another interesting aspect was to see the huge difference a properly cleaned dataset can make when using the same model.


## Outlook:
There are several ways how to improve these scores from my point of view. Some of them can be seen in other notebooks already (especially the larger ones).

1.   Run the code on several GPUs to figure out how well Rapids really performs in combination with DASK compared to one GPU or CPU.
2.   Ensemble different models based on their R2 score to test if the gap can be reduced this way (depending on the strength of the used models) An initial implementation for this is given in section 10. but I could not test it yet,
3.   Use larger grid searches to find better parameters for the given models
4.   An adaptive learning rate could also help with optimizing the results

## Limitations:
- It was possible to install the LBGM Version for GPU and the memory was consumed by the data but it did not run on the GPU somehow. So it was quite slow to do some training on this model and I could not compare it directly to XGB (or combine these two as intended in section 10)
- Some of the points mentioned in Outlook were not possible since running code in browser + only 36h GPU-time per week is hardly enough
- Automation of parameter search on large scale not feasible (not even with the GPU)


##References:
Inspirations for implementation and design:

- https://github.com/rapidsai/cuml/tree/branch-0.13/notebooks 
- https://www.kaggle.com/beniel/rapids-cudf-tutorial
- https://www.kaggle.com/aerdem4/rapids-svm-on-trends-neuroimaging
- https://www.kaggle.com/confirm/xfeat-cudf-lightgbm-catboost-wip
- https://www.kaggle.com/aiswaryaramachandran/baseline-model-xgboost-3-03-rmse/data?select=train_cleaned.csv
- https://www.kaggle.com/aiswaryaramachandran/eda-and-feature-engineering/comments
- https://www.kaggle.com/btyuhas/bayesian-optimization-with-xgboost
- https://www.kaggle.com/pnprabakaran/ny-city-taxi-fare-exploration-with-dask-and-dxgb
- https://www.kaggle.com/dsaichand3/lgbm-gpu
- https://www.kaggle.com/amar09/fare-prediction-stacked-ensemble-xgboost-lgbm/log
- https://www.kaggle.com/nicapotato/taxi-rides-time-analysis-and-oof-lgbm

Thanks to anyone whos code I have used!




# 0. Previous Commits - Comparison of different Models and Scores



Predictions are done when using the whole data set

### Linear Regression

**Commit 30 (Baseline) Score: 5.15497**
  - R2 score = 0.7166260480880737

### Ridge Regression

**Commit 31 Score: 5.15489**

params_ridge
  - alpha = loguniform(1e-5, 1e0)
  - solver = ['eig', 'cd']
  - n_iter = 100
  - cv = 5
  - verbose = 0
  - n_jobs = 1

ridge_params
  - alpha = 0.240960447726532
  - fit_intercept = True
  - normalize = False
  - solver = 'eig'

  - R2 score = 0.7166244983673096

### K-Nearest Neighbor 
**Commit 33 Score: 4.67805**
  - n_neighbors = 4
  - data_size = 55 Mio rows
  - R2 score = 0.7047221660614014

### Random Forest 
**Commit 34 Score: 3.96958**
  - n_estimators=10
  - n_jobs=-1
  - data_size = 55 Mio rows
  - R2 score = 0.6802817583084106

### XG Boost (GPU)
**Commit 5 Score: 3.98546**
  - n_estimators':500,
  - max_depth':7,
  - learning_rate':0.0005,
  - subsample':0.9,
  - colsample_bytree':0.9,
  - missing':-999,
  - random_state':2020,
  - objective':'reg:linear',
  - gamma':0.3,
  - reg_alpha':0.01,
  - tree_method':'gpu_hist'

  - data_size = 7 Mio. rows

### XG Boost (GPU) BEST
**Commit 23 Score: 3.19116**
  - n_estimators':350,
  - max_depth':7,
  - learning_rate':0.003,
  - subsample':0.9,
  - colsample_bytree':0.9,
  - missing':-999,
  - random_state':2020,
  - objective':'reg:linear',
  - gamma':0.3,
  - reg_alpha':0.01,
  - tree_method':'gpu_hist'  

  - rounds = 80000
  - early stopping = 250
  - R2 score = 0.832559
  - Test-rmse = 3.64371
  - Wall time = 49 min
  - data_size = 55 Mio. rows, cleaned dataset

# 1. Setup and Check Infrastructure
The first step is to get the infrastructure up and running. Important to mention here is that it is strongly recommended to use a GPU for calculation purpose.


If you find yourself running these notebooks on Colab as well as on Kaggle you might find this placeholder thing helpful. Only thing to touch is the environment you are running on (Kaggle or Colab).

Optional parameters:
- cleaned_dataset when you are running on Kaggle an use a precleaned one
- debug_mode for enabling a minimal row number
- rows_datasample defining how many rows will be used in the training (irrelevant when debug_mode=True)

In [ ]:
## Switch from Kaggle to Colab easily
environment='Kaggle' ## Kaggle / Colab

## Using Cleaned dataset? This can be useful when running into crashes or Cudf incompatibility, just add this to the kaggle input: https://www.kaggle.com/aiswaryaramachandran/baseline-model-xgboost-3-03-rmse/data?select=train_cleaned.csv
cleaned_dataset=True ## Change to True or False

## when True only 50.000 rows are used for debugging purpose. Set to False when doing real training
debug_mode=False ## Change to True or False

## choose how many rows of the training data sample you would like to use (only works when debug_mode=False ), max is 55423480
rows_datasample=5542348

### Installing all necessary librarys and tools

Setting up the environment path accordingly to your choice of environment

In [ ]:
if environment == 'Kaggle':
  env_submission_path='./'
  env_path='../input/new-york-city-taxi-fare-prediction/'
  print('The environment and paths were successfully setup for Kaggle')
elif environment == 'Colab':
  env_submission_path='/content/drive/My Drive/Colab Notebooks/'
  env_path='/content/drive/My Drive/Colab Notebooks/'

  from google.colab import drive
  drive.mount('/content/drive')

  print('The environment and paths were successfully setup for Colab')

else:
  print('Something went wrong here, please choose one of the options for path completion: Kaggle or Colab (or implement your own thing)')


Check for GPU

In [ ]:
!nvidia-smi

In [ ]:
!lscpu | grep "Model name:"
!lscpu | grep "CPU(s)"

Depending on the environment you might want to install different packages (or just link to them)

In [ ]:
%%time
if environment == 'Kaggle':
  import sys
  !cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
  !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
  sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
  sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
  sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
  !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
  print('You are all set for the kaggle rapids environment')

elif environment == 'Colab':
  # Install RAPIDS and Dask_ml
  !pip install dask_ml
  !pip install dask_cuda
  !git clone https://github.com/rapidsai/rapidsai-csp-utils.git
  !bash rapidsai-csp-utils/colab/rapids-colab.sh stable

  import sys, os
  dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
  sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
  sys.path
  exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())
  print('You are all set for the colab rapids environment')

else:
  print('Something went wrong here, please choose one of the options for path completion: Kaggle or Colab (or implement your own thing). If Kaggle failed please make sure you added the RAPIDS file on Kaggle to your Input!')

Library import, important to notice here is we are using mostly cuml librarys for GPU access

In [ ]:
import nvstrings
import numpy as np
import cudf, cuml
import dask_cudf
import io, requests
import math
import gc
import cupy as cp
import pandas as pd

import time 
import itertools

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns 

#Learning
from cuml.preprocessing.model_selection import train_test_split
from scipy.stats import uniform

# Linear Models https://github.com/rapidsai/cuml/tree/branch-0.13/notebooks
from cuml.linear_model import LinearRegression # Linear
from cuml.linear_model import LogisticRegression # Logisitc
from cuml.linear_model import ElasticNet # Elastic
from cuml.linear_model import Ridge # Ridge
from cuml.linear_model import Lasso # Lasso
from cuml.linear_model import MBSGDRegressor as cumlMBSGDRegressor # Mini Batch SGD Regressor

from cuml.solvers import SGD as cumlSGD # Stochastic Gradient Descent
from cuml.ensemble import RandomForestRegressor as cuRF # Random Forest
from cuml.dask.ensemble import RandomForestClassifier as cumlDaskRF # RandomForest

from cuml.neighbors import KNeighborsRegressor as cumlKNR # Nearest Neighbours
from cuml.svm import SVC # Support Vector Machines

from cuml import ForestInference
import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from cuml.metrics.regression import r2_score
from cuml.metrics.accuracy import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score as sk_acc
from sklearn.utils.fixes import loguniform

When running XG Boost distributed on several GPUs you need to install Dask_xgboost. Did this for some benchmarking, although it did not run on one GPU. You can spare this installation if you only have one GPU.

In [ ]:
#!pip install -U xgboost
!pip install dask-xgboost
import dask_xgboost as dxgb

import dask; print('Dask Version:', dask.__version__)

from dask.distributed import Client
from dask_cuda import LocalCUDACluster

# create a local cluster with 4 workers
n_workers = 1
# create a local CUDA cluster
cluster = LocalCUDACluster()
client = Client(cluster)
client

from dask.delayed import delayed

import dask.dataframe as dd
import dask.array as da

import cudf; print('cuDF Version:', cudf.__version__)
import numpy as np; print('NumPy Version:', np.__version__)

### Helper Functions

Since the save to CSV file and the calculation of the R2 score show up quite often during this kernel I decided to place them in small functions for an easy and reusable approach.

In [ ]:
def save_model_csv(y_pred, csv_name):
  save_submission = submission
  save_submission['fare_amount']=y_pred
  save_submission.to_csv(env_submission_path+csv_name, index=False)
  return save_submission.head(2)

In [ ]:
def calc_R2(y_pred, model_name):
  score = r2_score(y_pred, y_test)
  print("R2 Score for the "+model_name+" model is: ",score)
  return score

In [ ]:
## initializing the r2 scores so there will be no errors at the end when comparing them even if we do not run all models
score_linear_reg=0
score_ridge_reg=0
score_KNN=0
score_Rand_For=0
score_XGB_GPU=0
score_LGBM=0

# 2. First look at the Data (EDA)

The inital step to getting used to the data is having a look at it and loading it in a useable structure.

Before we can train something which predicts the taxi fares we first have to take a look what data we have available. We start by loading the data into cudf dataframes by selecting the relevant columns first. 

---
It happens that the system crashes if you choose a row number which is too large so that the data overflows your RAM / GPU (especially on colab, max 5 Mio rows). On Kaggle you can go up to max. 41 Mio. rows. If you choose to use the precleaned dataset you can use all the available data.

In [ ]:
cudf.set_allocator("managed")

## Check if using cleaned dataset or not
if cleaned_dataset==False:
    dtype = {     
        'key':'str',
        'fare_amount': 'float32',
        'pickup_datetime':'str',
        'pickup_longitude': 'float32',
        'pickup_latitude': 'float32',
        'dropoff_longitude': 'float32',
        'dropoff_latitude': 'float32',
        'passenger_count': 'int8'}
else:
    dtype = {     
        'key':'str',
        'fare_amount': 'float32',
        'pickup_datetime':'str',
        'pickup_longitude': 'float32',
        'pickup_latitude': 'float32',
        'dropoff_longitude': 'float32',
        'dropoff_latitude': 'float32',
        'passenger_count': 'int8',
        'pickup_latitude_round3': 'float32',
        'pickup_longitude_round3': 'float32',
        'dropoff_latitude_round3': 'float32',
        'dropoff_longitude_round3': 'float32',
        'is_pickup_JFK': 'int8',
        'is_dropoff_JFK': 'int8',
        'is_pickup_EWR': 'int8',
        'is_dropoff_EWR': 'int8',
        'is_pickup_la_guardia': 'int8',
        'is_dropoff_la_guardia': 'int8',
        'is_pickup_lower_manhattan': 'int8',
        'is_dropoff_lower_manhattan': 'int8'
    }

usecols = list(dtype.keys())

Load the data using cudf. Path variables are adjusted accordingly 

In [ ]:
%%time
# use a subset with 50.000 rows, max is nrows = 55423480

if debug_mode == True:
  ## using 1% (or how much you like)
  nrows = 50000
    
#del X_train, X_test, y_train, y_test
#gc.collect()

## Check if using cleaned dataset or not and adjusting the links accordingly
if cleaned_dataset==False:
    test = cudf.read_csv(env_path+'test.csv', usecols=usecols, dtype=dtype)
    train = cudf.read_csv(env_path+'train.csv', nrows=nrows, usecols=usecols, dtype=dtype)
else:
    nrows=55000000
    test = cudf.read_csv('../input/eda-and-feature-engineering/test_cleaned.csv', usecols=usecols, dtype=dtype)
    train = cudf.read_csv('../input/eda-and-feature-engineering/train_cleaned.csv', nrows=nrows, usecols=usecols, dtype=dtype)
    
submission = cudf.read_csv(env_path+'sample_submission.csv', usecols=usecols, dtype=dtype)

In [ ]:
train.head(5)

In [ ]:
test.head(5)

# 3. Data Cleaning

Before being able to start with the predictions, the data should be cleaned up and extended/enriched by relevant columns to achieve better results. For this, we are splitting the key (date & time) into seperate columns and add new features like the driven distance. This approch showed the best results so far. There are some limitations given that cudf and pandas are partly incompatible so I decided to test it out with a precleaned dataset as well to see the improvements later.

In [ ]:
#Drop Nan Values
train.nans_to_nulls()
train = train.dropna()

In [ ]:
#Checking shape of the data
print("Train: " + str(train.shape))
print("Test: " + str(test.shape))

Not available in Cudf datetime which could have improved results:
- Dayofyear, Weekofyear, Weekday, Quarter, day of month

In [ ]:
#Changing the data format of pickup_datetime and adding additional information about pickup time
train['pickup_datetime'] = train['pickup_datetime'].astype('datetime64[ns]')

train["hour"] = train.pickup_datetime.dt.hour
train["weekday"] = train.pickup_datetime.dt.weekday
train["month"] = train.pickup_datetime.dt.month
train["year"] = train.pickup_datetime.dt.year


test['pickup_datetime'] = test['pickup_datetime'].astype('datetime64[ns]')

test["hour"] = test.pickup_datetime.dt.hour
test["weekday"] = test.pickup_datetime.dt.weekday
test["month"] = test.pickup_datetime.dt.month
test["year"] = test.pickup_datetime.dt.year

Adding distances to airports does not work using cudf due to data type error (Series, Float) although the normal distance function (as below) works well. Tried different converting approaches including .astype(...) as you can see below at one example.

In [ ]:
def dist(pickup_lat, pickup_long, dropoff_lat, dropoff_long):  
    distance = np.abs(dropoff_lat - 40.7141667) + np.abs(dropoff_long - -74.0063889) ## not even working when putting the coordinates right in
    return distance

#calculate trip distance in miles
def calc_distance(data):

    # Distances to nearby airports, and city center
    # By reporting distances to these points, the model can somewhat triangulate other locations of interest
    nyc = (-74.0063889, 40.7141667)
    jfk = (-73.7822222222, 40.6441666667)
    ewr = (-74.175, 40.69)
    lgr = (-73.87, 40.77)
    data['distance_to_center'] = dist(40.7141667, -74.0063889, data['pickup_latitude'].astype(float, copy=False, errors='ignore'), data['pickup_longitude'].astype(float, copy=False, errors='ignore'))
    #data['pickup_distance_to_jfk'] = dist(jfk[1], jfk[0], data['pickup_latitude'], data['pickup_longitude'])
    #data['dropoff_distance_to_jfk'] = dist(jfk[1], jfk[0], data['dropoff_latitude'], data['dropoff_longitude'])
    #data['pickup_distance_to_ewr'] = dist(ewr[1], ewr[0],  data['pickup_latitude'], data['pickup_longitude'])
    #data['dropoff_distance_to_ewr'] = dist(ewr[1], ewr[0], data['dropoff_latitude'], data['dropoff_longitude'])
    #data['pickup_distance_to_lgr'] = dist(lgr[1], lgr[0],  data['pickup_latitude'], data['pickup_longitude'])
    #data['dropoff_distance_to_lgr'] = dist(lgr[1], lgr[0], data['dropoff_latitude'], data['dropoff_longitude'])
    
    data['long_dist'] = data['pickup_longitude'] - data['dropoff_longitude']
    data['lat_dist'] = data['pickup_latitude'] - data['dropoff_latitude']
    
    data['dist'] = dist(data['pickup_latitude'], data['pickup_longitude'],
                        data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data

#calc_distance(train)
#train.head(2)

In [ ]:
#calculate trip distance in miles
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [ ]:
train['distance'] = distance(train['pickup_latitude'], train['pickup_longitude'], train['dropoff_latitude'], train['dropoff_longitude'] )
test['distance'] = distance(test['pickup_latitude'], test['pickup_longitude'], test['dropoff_latitude'], test['dropoff_longitude'] )
train['distance'].describe()

In [ ]:
#check if everything worked
train.head(10)

In [ ]:
test.head(2)

In [ ]:
print("Ararage fare amount: " + str(train['fare_amount'].mean()))
print("Standard deviation fare amount: " + str(train['fare_amount'].std()))
print("Ararage distance: " + str(train['distance'].mean()) + " miles")
print("Standard deviation distance: " + str(train['distance'].std()) + " miles")

In [ ]:
train.describe()

Visualization of the data <br>

The following things were noticed (while using 500k datapoints):
*   The minimal fare_amount is negative. As this does not seem to be realistic I will drop them from the dataset.
*   Some of the minimum and maximum longitude/lattitude coordinates are way off. These  will also be remove from the dataset. (bounding box will be defined)
*   The average fare_amount is about 9.79 USD with a standard deviation of 7.48 USD. When building a predictive model we want to be better than 7.48 USD.



In [ ]:
train = train[train.fare_amount>=0]
train = train[(train['distance'] < 30) & (train['distance'] >=0 )]

In [ ]:
fare_amount = train['fare_amount'].to_array()
passenger_count = train['passenger_count'].to_array()
distance = train['distance'].to_array()

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(fare_amount).set_title("Verteilung des Fahrpreises")

In [ ]:
plt.figure(figsize=(8,5))
sns.kdeplot(distance).set_title("Distanz")

In [ ]:
#check max latitude und max longitude of test data
print("Max lat pickup: " + str(test['pickup_latitude'].max()))
print("Max lat dropoff: " + str(test['dropoff_latitude'].max()))
print("Max lon pickup: " + str(test['pickup_longitude'].max()))
print("Max lon dropoff: " + str(test['dropoff_longitude'].max()))
print("")
print("Min lat pickup: " + str(test['pickup_latitude'].min()))
print("Min lat dropoff: " + str(test['dropoff_latitude'].min()))
print("Min lon pickup: " + str(test['pickup_longitude'].min()))
print("Min lon dropoff: " + str(test['dropoff_longitude'].min()))

Bounding Box New York
<table>
  <tr>
    <th></th>
    <th>Dropoff</th>
    <th>Pickup</th>
  </tr>
  <tr>
    <td>Max Long</td>
    <td>-72.99096</td>
    <td>-72.986534</td>
   </tr>
   <tr>
    <td>Max Lat</td>
    <td>41.696682</td>
    <td>41.709553</td>
   </tr>
   <tr>
    <td>Min Long</td>
    <td>-74.26323</td>
    <td>-74.25219</td>
    </tr>
   <tr>
    <td>Min Lat</td>
    <td>40.568974</td>
    <td>40.57314</td>
   </tr>
</table>



In [ ]:
train.head(2)

In [ ]:
#Parts of train data are too far away, so they can be dropped
train = train[(train['pickup_longitude'] > -74.25) & (train['pickup_longitude'] < -72.98)]
train = train[(train['pickup_latitude'] > 40.57) & (train['pickup_latitude'] < 41.70)]
train = train[(train['dropoff_longitude'] < -72.99) & (train['dropoff_longitude'] > -74.26)]
train = train[(train['dropoff_latitude'] > 40.56) & (train['dropoff_latitude'] < 41.69)]

In [ ]:
dropoff_longitude = train['dropoff_longitude'].to_array()
dropoff_latitude = train['dropoff_latitude'].to_array()

city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

plt.figure(figsize=(10,6))
plt.scatter(dropoff_longitude, dropoff_latitude,
                color='green', 
                s=.02, alpha=.6)
plt.title("Dropoffs")

plt.ylim(city_lat_border)
plt.xlim(city_long_border)

Tested different columns to see which have the most impact. Pickup_datetime seems to be irrelevant for the predictions so dropping it together with key.

In [ ]:
## Kicking out unrelevant columns
unnecessary_columns=['key','pickup_datetime']
train=train.drop(unnecessary_columns,axis=1)
test=test.drop(unnecessary_columns,axis=1)

In [ ]:
train.head(2)

In [ ]:
test.head(2)

Splitting the train data here

In [ ]:
X=train.drop(['fare_amount'],axis=1)
y=train['fare_amount']

## getting the columns into the same order since XGB and LGBM seems to have a problem if they are not aligned
test = test[train.columns.drop(['fare_amount'])]

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## free some storage by cleaning up the previous unneeded variables
del X, y
gc.collect()

# 4. Linear Regression
This counts as our baseline model and showcases the most simple model here.

In [ ]:
lm = LinearRegression(fit_intercept = True, 
                      normalize = False,
                      algorithm = "eig")
lm.fit(X_train,y_train)

In [ ]:
y_test_pred=lm.predict(X_test)
score_linear_reg = calc_R2(y_test_pred, 'Linear Regression')

In [ ]:
y_pred_LR=lm.predict(test)

save_model_csv(y_pred_LR, 'submission_LinearReg')

# 5. Ridge Regression

This model is a little buggy and fails every second time to load. The results compared to the linear regression are not really an improvement here with R2 values and submission score being pretty much identical and still having longer training time.

In [ ]:
params_ridge = {
    "alpha": loguniform(1e-5, 1e0), # default 1.0
    "solver": ['eig', 'cd'], 
}
ridge = Ridge()
clf = RandomizedSearchCV(ridge, params_ridge, random_state=1, n_iter=100, cv=5, verbose=0, n_jobs=1)
best_model = clf.fit(X_train,y_train)

In [ ]:
best_model.best_estimator_.get_params()

In [ ]:
ridge_params = {
 'alpha': 0.240960447726532,
 'fit_intercept': True,
 'normalize': False,
 'solver': 'eig'
}

ridge = Ridge(**ridge_params)
result_ridge = ridge.fit(X_train,y_train)

In [ ]:
y_test_pred = result_ridge.predict(X_test)
score_ridge_reg =calc_R2(y_test_pred, 'Ridge Regression')

In [ ]:
y_pred_RR = result_ridge.predict(test)
save_model_csv(y_pred_RR, 'submission_RidgeReg')

# 6. K-Nearest Neighbors Regression



https://github.com/rapidsai/cuml/blob/branch-0.13/notebooks/kneighbors_regressor_demo.ipynb



In [ ]:
## params
n_neighbors=4

inspiration: https://www.kaggle.com/cdeotte/rapids-knn-30-seconds-0-938/notebook

In [ ]:
%%time
knn_cuml = cumlKNR(n_neighbors=n_neighbors)
knn_cuml.fit(X_train, y_train)

In [ ]:
y_test_pred = knn_cuml.predict(X_test)
score_KNN = calc_R2(y_test_pred, 'K-nearest Neigbors Regression')

In [ ]:
y_pred_KNN = knn_cuml.predict(test)

save_model_csv(y_pred_KNN, 'submission_KNN')

# 7. Random Forest GPU


In [ ]:
## params
n_estimators=10
n_jobs=-1

In [ ]:
%%time
rf_cuml = cuRF(n_estimators=n_estimators)
rf_cuml.fit(X_train, y_train)

In [ ]:
y_test_pred = rf_cuml.predict(X_test)
score_Rand_For =calc_R2(y_test_pred, 'Random Forest')

In [ ]:
y_pred_RF = rf_cuml.predict(test)
save_model_csv(y_pred_RF, 'submission_RandomForest')

# 8. XG Boost

In [ ]:
params_gpu = {
    'n_estimators':350,  
    'max_depth':7,    
    'learning_rate':0.003, 
    'subsample':0.9,    
    'colsample_bytree':0.9,    
    'missing':-999,    
    'random_state':2020,    
    'objective':'reg:linear',    
    'gamma':0.3,    
    'reg_alpha':0.01,    
    'tree_method':'gpu_hist'  # THE MAGICAL PARAMETER
}

## if you would like to compare runtime on CPU choose this one:
params_cpu = {
    'n_estimators':500,
    'max_depth':7,
    'learning_rate':0.0005,
    'subsample':0.9,
    'colsample_bytree':0.9,
    'missing':-999,
    'random_state':2020,
    'objective':'reg:linear',
    'gamma':0.3,
    'reg_alpha':0.01,
}

## XG Boost model GPU

In [ ]:
start_time=time.time()
timetaken_gpu_xgb=time.time()-start_time
timetaken_gpu_xgb

Running nicely in Kaggle but not in Colab
- TypeError: can not initialize DMatrix from DataFrame

In [ ]:
%%time
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=1000, 
                    early_stopping_rounds=100,evals=[(matrix_test,'test')])
    return model

XGB_model = XGBmodel(X_train,X_test,y_train,y_test,params_gpu)

- https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.DataFrame.to_delayed

- https://github.com/dask/dask-ml/issues/521

In [ ]:
timetaken_gpu=time.time()-start_time
print("GPU time taken for XG Boost model: ",timetaken_gpu_xgb)

In [ ]:
y_test_pred = XGB_model.predict(xgb.DMatrix(X_test))
score_XGB_GPU = calc_R2(y_test_pred, 'XGBoost GPU')

y_test_pred_xgb = y_test_pred

In [ ]:
y_pred_XGB = XGB_model.predict(xgb.DMatrix(test), ntree_limit = XGB_model.best_ntree_limit).tolist()

save_model_csv(y_pred_XGB, 'submission_XGboost_GPU')

## XG Boost Dask model for distributed clients (Not running with one client)

- https://www.kaggle.com/sandeepkumar121995/eda-data-cleaning-xg-boost
- https://www.kaggle.com/gunbl4d3/xgboost-ing-taxi-fares
- https://www.kaggle.com/aerdem4/m5-lofo-importance-on-gpu-via-rapids-xgboost
- https://www.kaggle.com/xhlulu/ieee-fraud-xgboost-with-gpu-fit-in-40s



Running into several problems with XG Boost. Python Kernel stops witout error message 	
- what(): parallel_for failed: cudaErrorNoKernelImageForDevice: no kernel image is available for execution on the device
- https://www.kaggle.com/c/otto-group-product-classification-challenge/discussion/13308
- https://github.com/CannyLab/tsne-cuda/issues/18

Hypothesis: There could be an error when loading a cudf dataset in XGBoost. Seems to not happen when using pandas

Error when using evals, turning it off results in Value Error. Not possible to convert matrix_test to something running, tested with pandas df, cudf, numpy. NaN cleanup again, still...
- https://github.com/dmlc/xgboost/issues/2274
- https://github.com/rapidsai/cudf/issues/2892

In [ ]:
%%time
def dask_XGBmodel(X_train,X_test,y_train,y_test,params):
    #matrix_test = xgb.DMatrix(data=dd.from_pandas(X_test.to_pandas(),1), label=dd.from_pandas(y_test.to_pandas(), 1))

    #Drop Nan Values
    X_test.nans_to_nulls()
    X_test = X_test.dropna()

    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model = dxgb.train(client, params=params, 
                       data=dd.from_pandas(X_train, 1), 
                       labels=dd.from_pandas(y_train, 1), 
                       num_boost_round=5000,
                       early_stopping_rounds=10,
                       evals=[(matrix_test,'test')]
                      )
    return model

dask_XGB_model = dask_XGBmodel(X_train,X_test,y_train,y_test,params)

#y_pred = dask_XGB_model.predict(testDGX)

> # 9. LGBM model (only runs on Kaggle and not really on GPU)

This section is just for showing that it runs. Due to the runtime bug that the training is not really running on the GPU it takes forever to train 1000 rounds and there is still a huge RMSE difference to XGB so I decided to focus more on XGB.

- https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

- https://www.kaggle.com/nicapotato/taxi-rides-time-analysis-and-oof-lgbm

- https://www.kaggle.com/dsaichand3/lgbm-gpu

- https://www.kaggle.com/aerdem4/rapids-svm-on-trends-neuroimaging


Compared CPU vs GPU:
- https://www.kaggle.com/ishivinal/sklearn-rapids-pandas


## Setup LGBM with GPU Support (only working on Kaggle, not Colab)

How to setup LGBM GPU Beta:
- https://www.kaggle.com/vinhnguyen/gpu-acceleration-for-lightgbm



In [ ]:
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM
!apt-get install -y -qq libboost-all-dev

In [ ]:
%%bash
cd LightGBM
rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

In [ ]:
!cd LightGBM/python-package/;python3 setup.py install --precompile
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
!rm -r LightGBM

In [ ]:
%%bash
cd ..

In [ ]:
#!pip install lightgbm --install-option=--gpu --force-reinstall

In [ ]:
import lightgbm as lgbm

In [ ]:
## Inspiration from https://www.kaggle.com/dsaichand3/lgbm-gpu
params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': 4,
        'num_leaves': 31,
        'learning_rate': 0.15,
        'max_depth': -1,
        'subsample': 0.8,
        'bagging_fraction' : 1,
        'max_bin' : 15,
        'bagging_freq': 20,
        'colsample_bytree': 0.6,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1,
        'zero_as_missing': True,
        'seed':0,
        'num_rounds':50000,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0
    }

In [ ]:
col_cat = X_train.select_dtypes('object').columns.tolist()

d_train = lgbm.Dataset(X_train.to_pandas(), label = y_train.to_pandas(), categorical_feature = col_cat, free_raw_data=False)
d_valid = lgbm.Dataset(X_test.to_pandas(), label = y_test.to_pandas(), categorical_feature = col_cat, free_raw_data=False)

Strange thing is, "GPU Memory consumption" is almost at limit with 41 Mio rows (15.1 GB, max. 15.9 GB) but "Cumulative GPU usage" still shows 0.00 %".

In [ ]:
%%time
model = lgbm.train(params_1, train_set = d_train, num_boost_round=10000, early_stopping_rounds=100, verbose_eval=10, valid_sets=d_valid)

In [ ]:
y_test_pred = model.predict(X_test, num_iteration = model.best_iteration)

score_LGBM = calc_R2(y_test_pred, 'LGBM GPU')

y_test_pred_lgbm = y_test_pred

In [ ]:
y_pred_LGBM = model.predict(test, num_iteration = model.best_iteration)

save_model_csv(y_pred_LGBM, 'submission_LGBM_GPU')

# 10. Stacked Ensemble XGB and LGBM

In my approach to further improve the predictions I tried to combine the two most advanced models as suggested by https://www.kaggle.com/amar09/fare-prediction-stacked-ensemble-xgboost-lgbm/log.

In [ ]:
## ensembled prediction over splitted test data, 50 / 50 split between XGB and LGBM
ensembled_prediction = (0.5*np.expm1(y_test_pred_xgb))+(0.5*np.expm1(y_test_pred_lgbm))

score_LGBM_XGB = =calc_R2(ensembled_prediction, 'LGBM XGB ensemble')

In [ ]:
## submitting the combined prediction, weighted 50 / 50
ensembled_prediction = (0.5*y_pred_XGB)+(0.5*y_pred_LGBM)

save_model_csv(ensembled_prediction, 'submission_ensemble.csv')

# 11. Evaluation

Show R2 Scores of the different models, makes it more convenient to compare in one graph.


In [ ]:
r2_score=[
    score_linear_reg,
    score_ridge_reg,
    score_KNN,
    score_Rand_For,
    score_XGB_GPU,
    score_LGBM,
    #score_LGBM_XGB
]

col={'R2 score':r2_score}
models=[
    'Linear Regression',
    'Ridge Regression',
    'KNN',
    'Random Forest',
    'XGB',
    'LGBM',
    #'LGBM & XGB'
]
df=pd.DataFrame(data=col,index=models)
df

df.plot(kind='bar')